In [1]:
from evoVAE.utils.datasets import MSA_Dataset
import evoVAE.utils.seq_tools as st
from sklearn.model_selection import train_test_split
import pandas as pd
from evoVAE.models.seqVAE import SeqVAE
import torch

## Data loading and preprocessing

In [2]:

# define threshold for sequence similarity 
THETA = 0.2

# Read in the datasets and create train and validation sets 
alns: pd.DataFrame = st.read_aln_file("../data/alignments/tiny.aln")
train, val = train_test_split(alns, test_size=0.2)

# create one-hot encodings and calculate reweightings 

# TRAINING 
train_encodings, train_weights = st.encode_and_weight_seqs(train["sequence"],theta=THETA)
train_ids = train["id"].values # just the seq identifiers 
train_dataset = MSA_Dataset(train_encodings, train_weights, train_ids)

# VALIDATION
val_encodings, val_weights = st.encode_and_weight_seqs(val["sequence"], THETA)
val_ids = val["id"].values
val_dataset = MSA_Dataset(val_encodings, val_weights, val_ids)


Encoding the sequences and calculating weights
The sequence encoding tensor has size: torch.Size([2, 5, 21])
The sequence weight array has size: (2,)

Encoding the sequences and calculating weights
The sequence encoding tensor has size: torch.Size([1, 5, 21])
The sequence weight array has size: (1,)



In [3]:
encoding, weights, id = train_dataset[0]
encoding.shape, weights, id

(torch.Size([5, 21]), 0.5, 'A0A0J8VL97_PhoQ_UniRef90')

In [4]:
translation = st.one_hot_to_seq(encoding)
translation

'MKLLL'

In [5]:
NUM_CHARS = 21
# get the sequence length 
seq_len = train_dataset[0][0].shape[0]
input_dims = seq_len * NUM_CHARS
latent_dims = 2


# use preset structure for hidden dimensions 
model = SeqVAE(input_dims=input_dims, latent_dims=latent_dims, hidden_dims=None) 

In [6]:
from random import shuffle


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=False)
test_loader = torch.utils.data.DataLoader(val_dataset, batch_size=20)
next(iter(train_loader))[0].shape,next(iter(train_loader))[1].shape


(torch.Size([2, 5, 21]), torch.Size([2]))

In [7]:
for encoding, weight, name in train_loader:
    outputs = model(encoding)
    print(outputs[0].shape)

    #elbo, kl, likelihood = model.loss_function(outputs, torch.flatten(encoding, start_dim=-1))
    #print(elbo, kl, likelihood)
    

ValueError: Value is not broadcastable with batch_shape+event_shape: torch.Size([2, 5, 21]) vs torch.Size([2, 105]).

In [ ]:
x_hat.shape